In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')
targetProb = pd.read_csv("gdrive/My Drive/BigDataBowl2021/targetProb.csv",
                 index_col=0)
qb_df = pd.read_csv("gdrive/My Drive/BigDataBowl2021/qb_data.csv",
                 index_col=0)

Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
targetProb = targetProb.drop(['targetNflId', 'dis', 'nflId'], axis=1)
targetProb.head()

,gameId,playId,gameClock,defendersInTheBox,passResult,time,x,y,s,a,event,epa,frameId,displayName,position,route,team,homeTeamAbbr,visitorTeamAbbr,targetedReceiver,team_name,qbX,qbY,distanceFromQB,angleToQB,openness,Position_FB,Position_HB,Position_RB,Position_TE,Position_WR,targetProb
0,2018090600,75,15:00:00,7.0,C,2018-09-07 01:07:18,39.31,8.423333,3.11,6.67,pass_forward,0.261827,36,Julio Jones,WR,HITCH,away,PHI,ATL,1.0,ATL,23.57,26.553333,24.009259,-49.036342,0.847541,0,0,0,0,1,0.96
1,2018090600,75,15:00:00,7.0,C,2018-09-07 01:07:18,37.35,18.773333,6.35,1.21,pass_forward,0.261827,36,Mohamed Sanu,WR,HITCH,away,PHI,ATL,0.0,ATL,23.57,26.553333,15.824563,-29.448491,0.799221,0,0,0,0,1,0.11
2,2018090600,75,15:00:00,7.0,C,2018-09-07 01:07:18,36.80,31.973333,6.21,2.68,pass_forward,0.261827,36,Austin Hooper,TE,OUT,away,PHI,ATL,0.0,ATL,23.57,26.553333,14.297178,22.277693,0.538622,0,0,0,1,0,0.03
3,2018090600,75,15:00:00,7.0,C,2018-09-07 01:07:18,30.38,34.943333,6.75,1.51,pass_forward,0.261827,36,Ricky Ortiz,FB,FLAT,away,PHI,ATL,0.0,ATL,23.57,26.553333,10.805934,50.934451,0.906936,1,0,0,0,0,0.26
4,2018090600,75,15:00:00,7.0,C,2018-09-07 01:07:19,38.89,5.463333,4.23,1.63,pass_arrived,0.261827,47,Julio Jones,WR,HITCH,away,PHI,ATL,1.0,ATL,23.70,25.813333,25.394066,-53.260985,0.163781,0,0,0,0,1,0.08


In [ ]:
qb_df = qb_df[['gameId', 'playId', 'displayName_QB']]
qb_df = qb_df.drop_duplicates()
qb_df.head()

,gameId,playId,displayName_QB
1,2018090600,75,Matt Ryan
60,2018090600,146,Matt Ryan
109,2018090600,168,Matt Ryan
157,2018090600,190,Matt Ryan
230,2018090600,256,Matt Ryan


In [ ]:
targetProb = targetProb.merge(qb_df,
                 how='left',
                 on=['gameId', 'playId'])
targetProb.head()
# OPENNESS BY ROUTE TYPE

,gameId,playId,gameClock,defendersInTheBox,passResult,time,x,y,s,a,event,epa,frameId,displayName,position,route,team,homeTeamAbbr,visitorTeamAbbr,targetedReceiver,team_name,qbX,qbY,distanceFromQB,angleToQB,openness,Position_FB,Position_HB,Position_RB,Position_TE,Position_WR,targetProb,displayName_QB
0,2018090600,75,15:00:00,7.0,C,2018-09-07 01:07:18,39.31,8.423333,3.11,6.67,pass_forward,0.261827,36,Julio Jones,WR,HITCH,away,PHI,ATL,1.0,ATL,23.57,26.553333,24.009259,-49.036342,0.847541,0,0,0,0,1,0.96,Matt Ryan
1,2018090600,75,15:00:00,7.0,C,2018-09-07 01:07:18,37.35,18.773333,6.35,1.21,pass_forward,0.261827,36,Mohamed Sanu,WR,HITCH,away,PHI,ATL,0.0,ATL,23.57,26.553333,15.824563,-29.448491,0.799221,0,0,0,0,1,0.11,Matt Ryan
2,2018090600,75,15:00:00,7.0,C,2018-09-07 01:07:18,36.80,31.973333,6.21,2.68,pass_forward,0.261827,36,Austin Hooper,TE,OUT,away,PHI,ATL,0.0,ATL,23.57,26.553333,14.297178,22.277693,0.538622,0,0,0,1,0,0.03,Matt Ryan
3,2018090600,75,15:00:00,7.0,C,2018-09-07 01:07:18,30.38,34.943333,6.75,1.51,pass_forward,0.261827,36,Ricky Ortiz,FB,FLAT,away,PHI,ATL,0.0,ATL,23.57,26.553333,10.805934,50.934451,0.906936,1,0,0,0,0,0.26,Matt Ryan
4,2018090600,75,15:00:00,7.0,C,2018-09-07 01:07:19,38.89,5.463333,4.23,1.63,pass_arrived,0.261827,47,Julio Jones,WR,HITCH,away,PHI,ATL,1.0,ATL,23.70,25.813333,25.394066,-53.260985,0.163781,0,0,0,0,1,0.08,Matt Ryan


In [ ]:
jose_df = targetProb[['gameId', 'playId', 'event', 'displayName', 'position', 'route', 'openness', 'displayName_QB', 'targetedReceiver', 'x', 'y']]
jose_df.head()

,gameId,playId,event,displayName,position,route,openness,displayName_QB,targetedReceiver,x,y
0,2018090600,75,pass_forward,Julio Jones,WR,HITCH,0.847541,Matt Ryan,1.0,39.31,8.423333
1,2018090600,75,pass_forward,Mohamed Sanu,WR,HITCH,0.799221,Matt Ryan,0.0,37.35,18.773333
2,2018090600,75,pass_forward,Austin Hooper,TE,OUT,0.538622,Matt Ryan,0.0,36.80,31.973333
3,2018090600,75,pass_forward,Ricky Ortiz,FB,FLAT,0.906936,Matt Ryan,0.0,30.38,34.943333
4,2018090600,75,pass_arrived,Julio Jones,WR,HITCH,0.163781,Matt Ryan,1.0,38.89,5.463333


In [ ]:
len(jose_df['playId'].unique())

4495

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
jose_df.to_csv('jose_df.csv')
!cp jose_df.csv "gdrive/My Drive/BigDataBowl2021/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
targetProb.targetedReceiver = targetProb.targetedReceiver.astype(int)
targetProb['alt_targetedReceiver'] = np.where(targetProb['targetedReceiver'] > .5, 'Targeted', 'Not Targeted')

In [ ]:
pass_forward = targetProb[targetProb.event == 'pass_forward']
pass_forward.head()
pass_arrived = targetProb[targetProb.event == 'pass_arrived']

In [ ]:
qb_summary_stats_pf = pass_forward[(pass_forward.alt_targetedReceiver == 'Targeted')].groupby('displayName_QB')\
  .agg(avg_targetProb = pd.NamedAgg(column='targetProb', aggfunc=np.average),
       count = pd.NamedAgg(column='displayName_QB', aggfunc='count'),
       targetProb_sd = pd.NamedAgg(column='targetProb', aggfunc=np.std),
       Average_Openness = pd.NamedAgg(column='openness', aggfunc=np.average),
       std_Openness = pd.NamedAgg(column='openness', aggfunc=np.std),
       Average_EPA = pd.NamedAgg(column='epa', aggfunc=np.average)
  )
qb_summary_stats_pf = qb_summary_stats_pf[qb_summary_stats_pf['count'] > 150]
qb_summary_stats_pf.reset_index(inplace=True)

In [ ]:
qb_summary_stats_pf.columns

Index(['displayName_QB', 'avg_targetProb', 'count', 'targetProb_sd',
       'Average_Openness', 'std_Openness', 'Average_EPA'],
      dtype='object')

In [ ]:
qb_summary_stats_pf.sort_values('Average_Openness')

,displayName_QB,avg_targetProb,count,targetProb_sd,Average_Openness,std_Openness,Average_EPA
3,Andy Dalton,0.742521,357,0.080861,0.647460,0.294265,0.123616
22,Josh Rosen,0.754290,366,0.081925,0.662636,0.304114,-0.177118
19,Jeff Driskel,0.753765,170,0.077757,0.669888,0.296610,0.014817
34,Ryan Fitzpatrick,0.737336,244,0.076744,0.669905,0.285326,0.220078
21,Josh Allen,0.746267,300,0.078948,0.674500,0.287205,-0.064523
20,Joe Flacco,0.751711,339,0.075338,0.680975,0.296094,0.042767
17,Jameis Winston,0.749088,373,0.075093,0.687390,0.296601,0.181681
36,Sam Darnold,0.748235,391,0.075906,0.688824,0.282246,-0.032678
27,Matthew Stafford,0.749364,535,0.079273,0.691932,0.289579,0.141869
23,Kirk Cousins,0.750072,554,0.078565,0.692481,0.291168,0.093620


In [ ]:
from google.colab import drive
qb_summary_stats_pf.to_csv('pf_sum_stats.csv')
!cp pf_sum_stats.csv "gdrive/My Drive/BigDataBowl2021/"

In [ ]:
qb_summary_stats_pf.reset_index(inplace=True)

In [ ]:
plot = sns.regplot(x='Average_Openness', y='Average_EPA', data=qb_summary_stats_pf,
            line_kws={'color':'red'}).set_title('Openness compared to EPA')
plot
# make another to be when each QB throws to the most open receiver as a % w/ QB labels as a bar chart

In [ ]:
play_data = pass_forward[(targetProb.gameId == 2018090600) & (targetProb.playId == 75)]

# snap_event = np.array(play_data[play_data.event == 'ball_snap']['time'])[0]

# play_data = play_data[play_data.time >= snap_event]

# skill_pos = ['WR','RB','TE','FB']

# skill_pos_df =  play_data[play_data.position.isin(skill_pos) ]


# player_name = skill_pos_df.displayName.unique()

# for player in player_name:
#     temp_df = skill_pos_df[skill_pos_df.displayName == player]
#     plt.scatter(50-temp_df.y,temp_df.x,s=(np.cbrt(temp_df.targetProb)/.45)**7)
# plt.axis('equal')

sns.scatterplot(x='x', y='y', data=play_data, size=np.cbrt(play_data['targetProb']/.45)**7, legend=False)
play_data

In [ ]:
pf_summary = pass_forward.groupby('alt_targetedReceiver')\
  .agg(avg_targetProb = pd.NamedAgg(column='targetProb', aggfunc=np.average),
       targetProb_sd = pd.NamedAgg(column='targetProb', aggfunc=np.std),
       avg_Openness = pd.NamedAgg(column='openness', aggfunc=np.average),
       std_Openness = pd.NamedAgg(column='openness', aggfunc=np.std)
  )
pf_summary

,avg_targetProb,targetProb_sd,avg_Openness,std_Openness
alt_targetedReceiver,,,,
Not Targeted,0.072945,0.054994,0.610594,0.297845
Targeted,0.752767,0.078467,0.708351,0.287324


In [ ]:
pa_summary = pass_arrived.groupby('alt_targetedReceiver')\
  .agg(avg_targetProb = pd.NamedAgg(column='targetProb', aggfunc=np.average),
       targetProb_sd = pd.NamedAgg(column='targetProb', aggfunc=np.std),
       avg_Openness = pd.NamedAgg(column='openness', aggfunc=np.average),
       std_Openness = pd.NamedAgg(column='openness', aggfunc=np.std)
  )
pa_summary

In [ ]:
qb_summary_stats_pa = pass_arrived[pass_arrived.alt_targetedReceiver == 'Targeted'].groupby('displayName_QB')\
  .agg(avg_targetProb_2 = pd.NamedAgg(column='targetProb', aggfunc=np.average),
       count_2 = pd.NamedAgg(column='displayName_QB', aggfunc='count'),
       targetProb_sd_2 = pd.NamedAgg(column='targetProb', aggfunc=np.std),
       avg_Openness_2 = pd.NamedAgg(column='openness', aggfunc=np.average),
       std_Openness_2 = pd.NamedAgg(column='openness', aggfunc=np.std),
       avg_epa_2 = pd.NamedAgg(column='epa', aggfunc=np.average)
  )
qb_summary_stats_pa = qb_summary_stats_pa[qb_summary_stats_pa['count_2'] > 100]
qb_summary_stats_pa.reset_index(inplace=True)

In [ ]:
qb_summary_stats_pa.head()

,displayName_QB,avg_targetProb_2,count_2,targetProb_sd_2,avg_Openness_2,std_Openness_2,avg_epa_2
0,Aaron Rodgers,0.203705,448,0.157156,0.533696,0.358118,0.473728
1,Alex Smith,0.196015,261,0.140727,0.458401,0.352029,0.188895
2,Andrew Luck,0.219818,548,0.167556,0.485579,0.357952,0.295093
3,Andy Dalton,0.227067,300,0.169090,0.473930,0.348736,0.332530
4,Baker Mayfield,0.220501,399,0.173109,0.521439,0.356236,0.304462


In [ ]:
qb_summary_stats_pc = pass_forward[pass_forward.alt_targetedReceiver == 'Targeted'].groupby('passResult')\
  .agg(avg_targetProb = pd.NamedAgg(column='targetProb', aggfunc=np.average),
       count = pd.NamedAgg(column='displayName_QB', aggfunc='count'),
       targetProb_sd = pd.NamedAgg(column='targetProb', aggfunc=np.std),
       Average_Openness = pd.NamedAgg(column='openness', aggfunc=np.average),
       std_Openness = pd.NamedAgg(column='openness', aggfunc=np.std),
       avg_epa = pd.NamedAgg(column='epa', aggfunc=np.average)
  )
qb_summary_stats_pc.reset_index(inplace=True)

In [ ]:
qb_summary_stats_pc['Pass_Result'] = np.where(qb_summary_stats_pc['passResult'] == 'C', 'Complete', 'Incomplete')
qb_summary_stats_pc

,passResult,avg_targetProb,count,targetProb_sd,Average_Openness,std_Openness,avg_epa,Pass_Result
0,C,0.764979,11154,0.079281,0.788253,0.250976,0.737552,Complete
1,I,0.729169,5188,0.071223,0.550080,0.290450,-0.797710,Incomplete
2,IN,0.718010,397,0.063757,0.531703,0.266646,-4.556730,Incomplete


In [ ]:
from google.colab import files
qb_summary_stats_pc.to_csv('comp_df.csv') 
!cp comp_df.csv "gdrive/My Drive/BigDataBowl2021/"

In [ ]:
passResult_df = pass_forward[pass_forward.targetedReceiver == 1][['passResult', 'openness']]
passResult_df.to_csv('results.csv') 
!cp results.csv "gdrive/My Drive/BigDataBowl2021/"

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.barplot(x="Pass_Result", y="Average_Openness", data=qb_summary_stats_pc).set_title('Difference in Openness by Pass Result')

In [ ]:
dummies = pd.get_dummies(pass_forward.passResult, prefix='passResult_')
pass_forward = pd.concat([pass_forward.reset_index(drop=True), dummies.reset_index(drop=True)], axis=1)
pass_forward.head()

In [ ]:
import statsmodels.api as smf

X = smf.add_constant(pass_forward[pass_forward.alt_targetedReceiver == 'Targeted']['openness'])
model = smf.Logit(pass_forward[pass_forward.alt_targetedReceiver == 'Targeted']['passResult__C'],
                  X).fit()
model.summary()

In [ ]:
import statsmodels.api as smf

X = smf.add_constant(pass_forward['openness'])
model = smf.Logit(pass_forward['targetedReceiver'],
                  X,
                  data=pass_forward[pass_forward.alt_targetedReceiver == 'Targeted']).fit()
model.summary()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Optimization terminated successfully.
         Current function value: 0.515846
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       targetedReceiver   No. Observations:                76552
Model:                          Logit   Df Residuals:                    76550
Method:                           MLE   Df Model:                            1
Date:                Sat, 21 Nov 2020   Pseudo R-squ.:                 0.01783
Time:                        17:41:20   Log-Likelihood:                -39489.
converged:                       True   LL-Null:                       -40206.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0251      0.023    -88.524      0.000      -2.070      -1.980
openness       1.1382      0.031     37.124      0.000       1.078       1.198
==============================================================================
"""

In [ ]:
logistic_model = pass_forward[['openness', 'targetedReceiver']]

In [ ]:
from google.colab import files
logistic_model.to_csv('logistic.csv') 
!cp logistic.csv "gdrive/My Drive/BigDataBowl2021/"

In [ ]:
def logit2prob(logit):
  odds = np.exp(logit)
  prob = odds / (1 + odds)
  return prob
logit2prob(1.3)

In [ ]:
qb_summary_combo = targetProb[targetProb.alt_targetedReceiver == 'Targeted'].groupby('event')\
  .agg(Average_Openness = pd.NamedAgg(column='openness', aggfunc=np.average))
qb_summary_combo.reset_index(inplace=True)
qb_summary_combo

,event,Average_Openness
0,pass_arrived,0.504263
1,pass_forward,0.708351


In [ ]:
from google.colab import files
qb_summary_combo.to_csv('event.csv') 
!cp event.csv "gdrive/My Drive/BigDataBowl2021/"

In [ ]:
sns.set_theme(style="whitegrid")
sns.barplot(x="event", y="Average_Openness", data=qb_summary_combo).set_title('Difference in Openness by Moment')

In [ ]:
qb_summary_combo = qb_summary_stats_pf.merge(qb_summary_stats_pa,
                                             how='inner',
                                             on='displayName_QB')
qb_summary_combo.head()

In [ ]:
qb_summary_combo['Diff_Openness'] = qb_summary_combo['Average_Openness'] - qb_summary_combo['avg_Openness_2']
qb_summary_combo.sort_values('Diff_Openness')
# % of time where the player targeted is when the receiver 